In [1]:
#Using Ollama 

Benefits:

No API charges - open-source
Data doesn't leave your box
Disadvantages:

Significantly less power than Frontier Model
Recap on installation of Ollama
Simply visit ollama.com and install!

Once complete, the ollama server should already be running locally.
If you visit:
http://localhost:11434/

You should see the message Ollama is running.

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter ollama serve
And in another Terminal (Mac) or Powershell (Windows), enter ollama pull llama3.2
Then try http://localhost:11434/ again.

If Ollama is slow on your machine, try using llama3.2:1b as an alternative. Run ollama pull llama3.2:1b from a Terminal or Powershell, and change the code below from MODEL = "llama3.2" to MODEL = "llama3.2:1b"

In [2]:
#Test the Ollama model

In [3]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [4]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [5]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [6]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [7]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources while maintaining consistency in their content.
2. **Product Design**: Generative AI can be used to design new products, packaging, and even entire brands. This can help businesses reduce the time and cost associated with designing new products.
3. **Marketing Automation**: Generative AI can be used to automate marketing tasks such as lead generation, email campaigns, and social media posting. This can help businesses scale their marketing efforts while improving efficiency.
4. **Customer Service Chatbots**: Generative AI can be used to create chatbots that can understand and respond to customer inquiries in a more human-like way. This can help businesses improve their customer service

In [9]:
# Introducing the ollama package
# And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

# Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [10]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Generative AI can create high-quality content such as articles, blog posts, social media posts, and even entire books. This can help businesses save time and resources on content creation.
2. **Product Design**: Generative AI can be used to design new products, such as fashion items, home decor, or electronic devices. It can also be used to redesign existing products with minimal human intervention.
3. **Image and Video Generation**: Generative AI can generate high-quality images and videos for various purposes, including marketing campaigns, advertising, and social media content.
4. **Chatbots and Customer Service**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequent questions and helping to resolve simple issues.
5. **Recommendation Systems**: Generative AI can be used to create personalized product recommendations for e-commerce website

In [11]:
#Alternative approach - using OpenAI python library to connect to Ollama

In [12]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Generative AI can create high-quality content such as articles, social media posts, and product descriptions, saving time and resources for content creators.
2. **Chatbots and Virtual Assistants**: Generative AI enables the development of more sophisticated chatbots and virtual assistants, providing personalized customer experiences and improving customer service.
3. **Image and Video Generation**: Generative AI can create realistic images and videos that can be used in various applications such as advertising, entertainment, and education.
4. **Predictive Maintenance**: Generative AI can analyze sensor data to predict equipment failures, reducing maintenance costs and improving overall efficiency.
5. **Personalized Recommendations**: Generative AI can generate personalized product recommendations for e-commerce businesses, enhancing customer experience and increasing sale

In [13]:
# Are you confused about why that works?
# It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

# Here's the scoop:

# The python class OpenAI is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.

# When you call openai.chat.completions.create(), this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

# Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

# OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

# So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions

# And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

# That's it!

# So when you say: ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
# Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

In [14]:
# Now let's try to make the same Scraper + Summarizer using Ollama3.2

In [15]:
# We already have the imports above so we don't need to get them again.

In [16]:
def get_website_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Remove scripts, styles, images, and inputs
    for tag in soup(['script', 'style', 'img', 'input']):
        tag.decompose()
    # Get clean text
    text = soup.get_text(separator="\n", strip=True)
    return text

In [17]:
system_prompt = "You are an assistant that summarizes website content. Respond in markdown."
user_prompt = "Summarize the following website:\n\n" + get_website_text("https://www.cnn.com/")

In [18]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [20]:
# Update the payload to use your messages
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [21]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

According to the provided text, here are some of the top news stories and topics:

1. **Politics**: 
	* Former senior Biden aide appears before House committee in probe of former president’s alleged mental decline.
	* Stanford's student newspaper sues Trump administration over use of immigration law to target pro-Palestinian students.
	* GOP Rep. Cory Mills accused by ex-girlfriend of threatening to release sexually explicit images, videos of her.
2. **Science and Discovery**:
	* Acting NASA chief fast-tracks plans to put a nuclear reactor on the moon.
3. **Entertainment and Culture**:
	* 'The Walking Dead' actress Kelley Mack dies at 33.
4. **Sports**:
	* Tennis coach Patrick Mouratoglou opens up on recent split with Naomi Osaka.

These are just a few of the many topics covered in the provided text.
